# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [9]:
#Extracting the columns we need for our analysis
callback_data = data[['race','call']]
callback_data.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


### Initial observation from the data

In [11]:
print('Number of resumes sent out with black sounding names (b) and white sounding names (w):')
print(callback_data['race'].value_counts())

Number of resumes sent out with black sounding names (b) and white sounding names (w):
b    2435
w    2435
Name: race, dtype: int64


In [20]:
#Creating an array with the results from the resume with white sounding names
white_filter = callback_data['race'] == 'w'
callback_data_white = callback_data.loc[white_filter]
white_boolean = np.array(callback_data_white['call'])

print('Percentage of callbacks for the resume with a white sounding name:')
print((np.sum(white_boolean)) / len(white_boolean) * 100,'%')


Percentage of callbacks for the resume with a white sounding name:
9.65092402464 %


In [21]:
#Creating an array with the results from the resume with black sounding names
black_filter = callback_data['race'] == 'b'
callback_data_black = callback_data.loc[black_filter]
black_boolean = np.array(callback_data_black['call'])

print('Percentage of callbacks for the resume with a black sounding name:')
print((np.sum(black_boolean)) / len(black_boolean) * 100,'%')

Percentage of callbacks for the resume with a black sounding name:
6.4476386037 %


In [48]:
#Computing the observed percentage difference
print('Observed difference between the callback rate from an identical resume with a white and black sounding name:')
print(observed_stat,'%')

Observed difference between the callback rate from an identical resume with a white and black sounding name:
3.20328542094 %


### Testing our hypothesis using permutation resampling 

We see above from the data that identical resumes with white sounding names get approximately 3% more callbacks than the same resume with a black sounding name.

To test if race did play a part in this discripency of callbacks and it was not due to just chance, we will conduct a hypothesis test using permutation resampling.

**Null hypothesis:** The difference in callback rate was due to chance<br>
**Alternative hypothesis:** The difference in callback rate was due to difference in white and black sounding names

For our hypothesis test we will use permutation resampling to stimulate the collection of data under the null hypothesis which is that the difference in callback rates was completely random.

Our test statistic here is the percentage difference in callback rates recieved by an indentical resume with a black sounding name and a white sounding name

Using permutation resampling we will generate 100,000 permutation replicates(our t-statistic) to compare with the observed statistic of 3%

In [45]:
#Function that calculates our t-stat (percentage difference of successes between 2 arrays)
def t_stat(x,y):
    frac_x = np.sum(x) / len(x)
    frac_y = np.sum(y) / len(y)
    return (frac_x - frac_y) * 100

observed_stat = t_stat(white_boolean,black_boolean)

merged_array = np.concatenate((white_boolean,black_boolean),axis=0)

perm_replicates = np.empty(100000)

#Generating 100,000 permutation replicates(our t-stat) to test our hypothesis
for i in range(100000):
    perm_resample = np.random.permutation(merged_array)
    w_perm = perm_resample[:len(white_boolean)]
    b_perm = perm_resample[len(white_boolean):]
    t_statistic = t_stat(w_perm,b_perm)
    perm_replicates[i] = t_statistic

#Computing the 95% confidence intervals
print('95% Confidence Intervals:')
print(np.percentile(perm_replicates,[2.5,97.5]))

#Computing the margin of error using the standard error 
standard_error = np.std(perm_replicates) / np.sqrt(len(perm_replicates))
MOE = observed_stat * standard_error
print('Margin of Error:')
print(MOE)

#Computing our p-value
p_value = np.sum(perm_replicates >= observed_stat) / len(perm_replicates)
print()
print()
print('P-value:')
print(p_value)
print('i.e 0.00004')

95% Confidence Intervals:
[-1.56057495  1.56057495]
Margin of Error:
0.00788701268048


P-value:
4e-05
i.e 0.00004


Conclusion: 95% of values of the test-statistic from random distribution (percentage difference in callback) lie between -1.5605% and 1.5605% with a margin of error of 0.0078%. 

Our p-value is 0.00004, which is statistically significant because it indicates the percentage difference in callback rates from random distribution was as high as the observed difference of 3% only 0.004% of the time.

As a result we can reject our null hypothesis and conclude that the difference in callback rates in identical resume is not due to random chance, but race/name does play a part in the amount of callback a resume recieves.

**Warning: This data does not conclude that race/name is the most important part in callback success rate of a resume.** **This data simply concludes that given two exactly indentical resumes, one with a black sounding name** **and one with a white sounding name, the one with the white sounding name will recieve approximately 3% more** **callbacks than the same resume with a black sounding name.**
